In [48]:
import sys

# Select path to current folder and split by \\
main_path = sys.path[0].split("\\")

# Asssign path to parent folder
# path_to_parent allows access to any folder from within parent folder, no matter the location of this file within the parent folder
# i.e.: Don't need to specify "../" x amount of times
path_to_parent = []
for element in main_path:
    path_to_parent.append(element)
    if "Fake_Users_Movies_Classifier" == element:
        break

path_to_parent = "\\".join(path_to_parent)

# Add path to feature generation folder
sys.path.append(path_to_parent+"\\feature_generation")

In [49]:
# Import feature generator for week 1
from feature_gen_wk3 import feature_gen

# Create string path to labelled data
path_to_file = path_to_parent + "/labelled_data/first_batch_with_labels_likes.npz"
# Generate features from file
df_final = feature_gen().retrieveAndGenerate(path_to_file)

In [50]:
data, yy = feature_gen().extractDataFromNPZ(path_to_file)

In [51]:
import pandas as pd
import numpy as np

In [52]:
df_base = data


# Sort data by item, remove user column and calculate mean rating of each item
XX = df_base.sort_values(by=["item"], ascending=True)
XX = XX.drop(['user'],axis=1)
XX = XX.groupby(by='item').aggregate('mean')

# Retrieve user ID's
users = df_base['user'].unique()

# Initialize empty list
users_mean_diff = []
# Retrieve items rated by users and convert to dictionary
user_items = df_base.groupby('user')['item'].agg(list).to_dict()

# Iterate through all users
for i in users:
    # Retrieve items that the user rated (interacted with)
    items = user_items[i]
    # Set mean_ID_iff to 0
    mean_ID_diff=0

    # Loop through all items
    for j in range(0,len(items)-1):
        # Calculate the difference between ID of current item, and the next item
        ID_diff = items[j+1]-items[j]
        # Increment mean_diff by value of ID_diff
        mean_ID_diff += ID_diff

    # Divide mean_ID_diff by total amount of items to calculate mean
    mean_ID_diff /= len(items)
    # Create a dictionary of user : mean item ID difference pair
    user_mean_diff = {'user':i,'mean_ID_diff':mean_ID_diff}
    # Append dictionary to list
    users_mean_diff.append(user_mean_diff)

# Convert list to pandas dataframe and sort values by user ID
users_mean_diff = pd.DataFrame(users_mean_diff)
users_mean_diff = users_mean_diff.sort_values(by='user').reset_index(drop=True)

In [54]:
f = lambda row: -1 if row < -0.5 else 0
XX['mode'] = XX['rating'].apply(lambda row: 1 if row > 0.5 else f(row))

In [59]:
df = data

In [81]:
items

item      [15, 19, 21, 31, 46, 49, 54, 55, 56, 58, 73, 7...
rating    [0, 1, 0, 0, 1, 0, 1, -1, 0, 1, -1, 0, 0, 0, -...
Name: 0, dtype: object

In [94]:
users_item_ratings = df.groupby('user').agg(list)


user_item_comp = []
for user in users_item_ratings.index:
    items = users_item_ratings.loc[user]['item']
    ratings = users_item_ratings.loc[user]['rating']

    item_comp = {'user':i,'modal_ratings':0,'non_modal_ratings':0,'rating_diff':[]}

    items_mode = XX['mode'][items]
    
    item_comp['modal_ratings'] = np.sum(ratings == items_mode)
    item_comp['rating_diff'] = abs(items_mode - ratings)

    item_comp['non_modal_ratings'] = len(item_comp['rating_diff']) - item_comp['modal_ratings']
    user_item_comp.append(item_comp)

In [92]:
set(abs(items_mode - ratings))

{0, 1, 2}

In [73]:
user_item_ratings = df.groupby('user').agg(list)

In [58]:
df = data



df_new = pd.DataFrame(user_item_comp).sort_values(by='user').reset_index(drop=True)
df_new['diff_std'] = df_new['rating_diff'].apply(lambda row: np.std(row))
df_new['diff_mean'] = df_new['rating_diff'].apply(lambda row: np.mean(row))

mode_ratings_diff = []
for index,row in df_new.iterrows():
    x = row['modal_ratings']
    y = row['non_modal_ratings']
    diff = min(x,y)/max(x,y)
    mode_ratings_diff.append(diff)
df_new['mode_ratings_diff'] = mode_ratings_diff
df_new = df_new.sort_values(by='user').reset_index(drop=True)
df_new = df_new.join(users_mean_diff['items_diff'])

df_new = df_new.drop(['rating_diff'],axis=1)
df_new.index = df_new['user']

KeyError: 'items_diff'

In [47]:
df2['rating_diff']

KeyError: 'rating_diff'

In [ ]:
df.drop([i for i in range(0,999)],axis=1).columns

In [ ]:
allData = pd.DataFrame()

for XX in [XX1,XX2,XX3,XX4]:
    user_features = XX.groupby('user')['item'].agg(
        mean_item_id='mean',
        median_item_id='median',
        item_id_variance='var',
        item_id_std_dev='std',
        item_id_range=lambda x: x.max() - x.min(),
        skewness=skew,
        kurtosis=kurtosis,
        interaction_polarity=lambda x: x.mean() - x.median(),
        mean_difference=lambda x: x.diff().mean(),
        max_difference=lambda x: x.diff().max()
    ).reset_index()
    
    user_features = user_features.sort_values(by='user')
    user_features.index = user_features['user']

    user_features["IQR7525"] = XX.groupby('user')['rating'].apply(lambda x: x.quantile(0.75) - x.quantile(0.25))
    user_features["IQR8020"] = XX.groupby('user')['rating'].apply(lambda x: x.quantile(0.80) - x.quantile(0.20))

    user_features["rating_entropy"] = XX.groupby('user')['rating'].apply(lambda x: entropy(x.value_counts(normalize=True)))
    allData = pd.concat([allData,user_features])

In [ ]:
labels2.index = labels2.user
labels3.index = labels3.user
labels4.index = labels4.user

In [ ]:
labels = pd.concat([labels1,labels2,labels3,labels4])
labels.index = labels.user
labels = list(labels['label'])

In [ ]:
allData['label'] = labels
allData

In [ ]:
allData.to_csv("./csv_files/perfect_feat_all.csv")

In [ ]:
data = np.load("fifth_batch_likes.npz")

In [ ]:
XX, label = eD.simpleDataRetrieval(data)

In [ ]:
XX

In [ ]:
user_features = XX.groupby('user')['item'].agg(
        mean_item_id='mean',
        median_item_id='median',
        item_id_variance='var',
        item_id_std_dev='std',
        item_id_range=lambda x: x.max() - x.min(),
        skewness=skew,
        kurtosis=kurtosis,
        interaction_polarity=lambda x: x.mean() - x.median(),
        mean_difference=lambda x: x.diff().mean(),
        max_difference=lambda x: x.diff().max()
    ).reset_index()
user_features.index = user_features['user']
user_features["IQR7525"] = XX.groupby('user')['rating'].apply(lambda x: x.quantile(0.75) - x.quantile(0.25))
user_features["IQR8020"] = XX.groupby('user')['rating'].apply(lambda x: x.quantile(0.80) - x.quantile(0.20))

user_features["rating_entropy"] = XX.groupby('user')['rating'].apply(lambda x: entropy(x.value_counts(normalize=True)))

In [ ]:
user_features

In [ ]:
user_features.to_csv("./csv_files/perfect_feat_fifth.csv")